In [19]:
import importlib
import stockInfo
from stockInfo import create_and_compile_model
importlib.reload(stockInfo)
from stockInfo import fetch_stock_data, dataCollect
from tabulate import tabulate


tick = input('What is your Ticker Name?').upper()
start_date = input('Enter start date (YYYY-MM-DD):')
end_date = input('Enter end date (YYYY-MM-DD):')

print('Entered Ticker:', tick)

stock_result = fetch_stock_data(tick, start_date, end_date)

if stock_result['company_name'] != 'N/A' and stock_result['data']:
    print('Company Name:', stock_result['company_name'])
    print('Stock Data:')
    for col, values in stock_result['data'].items():
        print(f'{col}: {values}')
else:
    print('Error')


Entered Ticker: TSLA
[*********************100%%**********************]  1 of 1 completed
Company Name: Tesla, Inc.
Stock Data:
Open: [218.60000610351562, 208.68666076660156, 200.18333435058594, 202.72666931152344, 233.43333435058594, 233.13333129882812, 223.3333282470703, 231.3633270263672, 234.4499969482422, 218.9566650390625, 228.09666442871094, 215.53334045410156, 228.19667053222656, 225.2566680908203, 222.6366729736328, 204.3333282470703, 213.9566650390625, 205.2133331298828, 200.5833282470703, 215.5399932861328, 229.4566650390625, 235.90333557128906, 230.10000610351562, 229.0, 225.7933349609375, 225.92333984375, 228.56666564941406, 237.56666564941406, 256.8999938964844, 247.6999969482422, 242.88333129882812, 239.86666870117188, 239.13999938964844, 234.92333984375, 247.1666717529297, 239.93333435058594, 247.0, 239.32000732421875, 232.1366729736328, 233.1699981689453, 222.52999877929688, 234.60000610351562, 226.31333923339844, 227.02000427246094, 226.9199981689453, 221.933334350585

In [20]:
if stock_result is not None:
    print('Stock Data Tail:')
    for col, values in stock_result['data'].items():
        print(f'{col}: {values[-5:]}')
else:
    print('Error')

Stock Data Tail:
Open: [235.75, 233.8699951171875, 242.9199981689453, 241.5500030517578, 240.27000427246094]
High: [239.3699951171875, 246.66000366210938, 246.57000732421875, 244.0800018310547, 245.27000427246094]
Low: [233.2899932861328, 233.6999969482422, 239.1699981689453, 236.97999572753906, 239.27000427246094]
Close: [235.5800018310547, 238.72000122070312, 239.3699951171875, 242.63999938964844, 243.83999633789062]
Adj Close: [235.5800018310547, 238.72000122070312, 239.3699951171875, 242.63999938964844, 243.83999633789062]
Volume: [104099800, 137971100, 126436200, 107142300, 102980100]


In [6]:
data_collect = dataCollect(yahoo_info)
stock_data_df = data_collect.compress_data_to_df()
if stock_data_df is not None:
    print(stock_data_df.head())
    print(stock_data_df.tail())

[*********************100%%**********************]  1 of 1 completed


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-03-04  218.600006  222.816666  200.000000  207.146667  207.146667   
2021-03-05  208.686661  209.279999  179.830002  199.316666  199.316666   
2021-03-08  200.183334  206.710007  186.263336  187.666672  187.666672   
2021-03-09  202.726669  226.029999  198.403336  224.526672  224.526672   
2021-03-10  233.433334  239.283340  218.353333  222.686661  222.686661   

               Volume  
Date                   
2021-03-04  197758500  
2021-03-05  268189500  
2021-03-08  155361000  
2021-03-09  202569900  
2021-03-10  181817100  
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-12-04  235.750000  239.369995  233.289993  235.580002  235.580002   
2023-12-05  233.869995  246.660004  233.699997  238.720001  238.720001   
2023-12-06  242.

In [7]:
if stock_data_df is not None:
    model, rmse = create_and_compile_model(stock_data_df)
    print(model.summary())
    print('Root Mean Squared Error (RMSE): ', rmse)
    

Epoch 1/7
499/499 [==============================] - 19s 26ms/step - loss: 0.0176 - mean_absolute_error: 0.0978
Epoch 2/7
499/499 [==============================] - 12s 24ms/step - loss: 0.0049 - mean_absolute_error: 0.0530
Epoch 3/7
499/499 [==============================] - 13s 27ms/step - loss: 0.0035 - mean_absolute_error: 0.0458
Epoch 4/7
499/499 [==============================] - 12s 24ms/step - loss: 0.0029 - mean_absolute_error: 0.0411
Epoch 5/7
499/499 [==============================] - 12s 23ms/step - loss: 0.0024 - mean_absolute_error: 0.0388
Epoch 6/7
499/499 [==============================] - 12s 24ms/step - loss: 0.0022 - mean_absolute_error: 0.0365
Epoch 7/7
5/5 [==============================] - 1s 28ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                

In [8]:
import math
data = stock_data_df.filter(['Close'])
dataset = data.values
#Get the number of rows to train the model on
training_data_len = math.ceil(len(dataset) * .8)

training_data_len

559

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)